# Working Version From Machinelearning mastery

In [1]:
# SG Sigmoid in Generator 
# WD Weak Discriminator
# SL Soft Labels
# GN Gaussian Noise (not uniform)
# OHE One Hot Encoding
# Replaced Concatenate with multiply

# TODO
# basic conditional gan
# generate data
# add and train a classifier use k fold also
# condition me one hot encoding
# generator k lie combined data and disc k lie train data.. coz testing me sara deke bias ho jaega
# different loss functions for generator and discriminator
# discriminator output shud be one hot encode with labels as n s or v and not 0 or 1 that is fake or real:
# confusion here
# disc property: should recognise fake and also if gen data is not of that class (so 2 things)
# training k time jab random samples uthaenge tab sari classes k
# equal no. of samples uthana

In [2]:
%config Completer.use_jedi = False

# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"] = ""

# example of training an conditional gan on the fashion mnist dataset
from numpy import expand_dims
from numpy import zeros
from numpy import ones
import keras
from numpy.random import randn
from numpy.random import randint
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input, Reshape, multiply, Embedding, merge, Concatenate, Conv1D, BatchNormalization
from keras.layers import Dense, Flatten, Multiply
from keras.layers.advanced_activations import LeakyReLU
import numpy as np
from keras.utils import to_categorical
import os
from numpy import asarray
import matplotlib.pyplot as plt

In [3]:
# in_label = Input(shape=(input_classes,))
# x = Embedding(input_classes, 30)(in_label)
# x = Dense(int(data_dim/input_classes))(x)
# x = Reshape((data_dim,1))(x) 

In [4]:
def discriminator(data_dim, input_classes=3):
    
#     in_label = Input(shape=(1,))
#     x = Embedding(input_classes, 30)(in_label)
#     x = Dense(data_dim)(x)
#     x = Reshape((data_dim,1))(x)
    
    in_label = Input(shape=(input_classes,))
    x = Embedding(input_classes, 30)(in_label)
    x = Dense(int(data_dim/input_classes))(x)
    x = Reshape((data_dim,1))(x)
    
    D_in = Input(shape=[data_dim,1])
    x = Multiply()([D_in, x])
#     x = Concatenate()([D_in, x])
    
    x = Conv1D(filters=32, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1D(filters=32*2, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1D(filters=32*4, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1D(filters=32*8, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1D(filters=32*16, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Flatten()(x)
    out = Dense(1, activation='sigmoid')(x)
    
#     model = Model(inputs=[D_in, in_label], outputs=out)
#     opt = Adam(lr=0.0002, beta_1=0.5)
#     loss = 'binary_crossentropy'
#     model.compile(loss=loss, optimizer=opt, metrics=['accuracy'])
#     return model
    
    out1 = Dense(input_classes, activation='softmax')(x)
    
    model = Model(inputs=[D_in, in_label], outputs=[out, out1])
    opt = Adam(lr=0.0002, beta_1=0.5)
    loss = ['binary_crossentropy', 'categorical_crossentropy'] # 'sparse_categorical_crossentropy'
    model.compile(loss=loss, optimizer=opt, metrics=['accuracy'])
    return model

# d_model = discriminator(data_dim=186, input_classes=3)
# plot_model(d_model, to_file='disc.png', show_shapes=True)

In [5]:
# noise_dim=100
# in_label = Input(shape=(input_classes,))
# print (in_label.shape)
# x = Embedding(1, 30)(in_label)
# print (x.shape)
# x = Dense(100)(x)
# print (x.shape)
# x = Reshape((noise_dim,1))(x)    
# print (x.shape)

In [6]:
from keras.layers import UpSampling1D
from keras.layers.core import Activation
from keras.layers import Add
import tensorflow as tf

def generator(noise_dim=99, input_classes=3, out_dim=186):
    
#     in_label = Input(shape=(1,))
#     x = Embedding(input_classes, 30)(in_label)
#     x = Dense(noise_dim)(x)
#     x = Reshape((noise_dim,1))(x)
    
    in_label = Input(shape=(input_classes,))
    x = Embedding(input_classes, 30)(in_label)
    x = Dense(int(noise_dim/input_classes))(x)
    x = Reshape((noise_dim,1))(x)    
    
    G_in = Input(shape=[noise_dim,])
    gen = Dense(noise_dim)(G_in)
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = Reshape((noise_dim,1))(gen)

    x = Multiply()([gen, x])
#     x = Concatenate()([gen, x])
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*16, kernel_size=13, strides=1, padding='valid', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = UpSampling1D()(x)
    x = Conv1D(filters=32*8, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*8, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*4, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*4, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*2, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=1, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    out = Activation('sigmoid')(x)  # for tanh change here
    model = Model(inputs=[G_in, in_label], outputs=out)

    return model

# g_model = generator(noise_dim=99, input_classes=3, out_dim=186)
# plot_model(g_model, to_file='gen.pdf', show_shapes=True)

In [7]:
def create_gan(d_model, g_model):
    
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # get noise and label inputs from generator model
    gen_noise, gen_label = g_model.input
    # get image output from the generator model
    gen_output = g_model.output
    # connect image output and label input from generator as inputs to discriminator
    gan_output = d_model([gen_output, gen_label])
    # define gan model as taking noise and label and outputting a classification
    model = Model([gen_noise, gen_label], gan_output)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
#     loss = 'binary_crossentropy'
#     model.compile(loss=loss, optimizer=opt)
#     return model
    loss = ['binary_crossentropy', 'categorical_crossentropy'] # 'sparse_categorical_crossentropy'
    model.compile(loss=loss, optimizer=opt)
    return model

# gan_model = create_gan(d_model, g_model)
# plot_model(gan_model, to_file='gan.pdf', show_shapes=True)

In [8]:
def load_real_samples():
    X = np.load('Data/X.npy')
    y = np.load('Data/y.npy')

    # print (X.shape, y.shape)

    X_N = X[y==0]
    X_S = X[y==1]
    X_V = X[y==2]

    y_N = y[y==0]
    y_S = y[y==1]
    y_V = y[y==2]

    # print (X_N.shape, y_N.shape)
    # print (X_S.shape, y_S.shape)
    # print (X_V.shape, y_V.shape)

    X_N=X_N.reshape(X_N.shape[0],X_N.shape[1],1)
    X_S=X_S.reshape(X_S.shape[0],X_S.shape[1],1)
    X_V=X_V.reshape(X_V.shape[0],X_V.shape[1],1)

    # print (X_N.shape, y_N.shape)
    # print (X_S.shape, y_S.shape)
    # print (X_V.shape, y_V.shape)
    return X_N, y_N, X_S, y_S, X_V, y_V

In [9]:
def generate_real_samples(X_N, y_N, X_S, y_S, X_V, y_V, n_samples):
    
    # choose random instances
    i_N = randint(0, y_N.shape[0], int(n_samples/3))
    i_S = randint(0, y_S.shape[0], int(n_samples/3))
    i_V = randint(0, y_V.shape[0], int(n_samples/3))
    
    # select ECG and labels
    X = np.vstack((X_N[i_N], X_S[i_S], X_V[i_V]))
    labels = np.hstack((y_N[i_N], y_S[i_S], y_V[i_V]))
    labels = keras.utils.to_categorical(labels)
    # print (labels.shape)
    
    # generate class labels
    y = np.random.uniform(0.7, 1, n_samples)
    y = y.reshape(y.shape[0], 1)
#     y = np.ones((n_samples, 1))
    return [X, labels], y

In [10]:
def reshape(X):
    if len(X.shape) == 1:
        X = X.reshape(X.shape[0], 1)
        return X
    else:
        if X.shape[-1] == 1:
            return X
        else:
            X = X.reshape(X.shape[0], X.shape[1], 1)
            return X

# generate points in latent space as input for the generator
# normal noise
def generate_latent_points(latent_dim, n_samples, n_classes=3):
    # generate points in the latent space
#     X_fake = np.random.uniform(0, 1.0, size=[n_samples, latent_dim])
    X_fake = np.random.normal(0,1.0,(n_samples,latent_dim))
    # generate labels
    labels_fake = np.hstack((np.zeros(int(n_samples/3)), np.ones(int(n_samples/3)), 2*np.ones(int(n_samples/3))))
    np.random.shuffle(labels_fake)
    labels_fake = keras.utils.to_categorical(labels_fake)
    return [reshape(X_fake), labels_fake]

In [11]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
    # generate points in latent space
    z_input, labels_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    images = generator.predict([z_input, labels_input])
    # create class labels
    y = np.random.uniform(0, 0.3, n_samples)
    y = y.reshape(y.shape[0], 1)
#     y = np.zeros((n_samples, 1))
    return [images, labels_input], y

In [12]:
# create and save a plot of generated images
def save_plot(X, n):
    plt.figure(figsize=(10,3))
    for i in range(n * n):
        # define subplot
        plt.subplot(n, n, 1 + i)
        # turn off axis
        plt.axis('off')
        # plot raw pixel data
        plt.plot(X[i, :, 0])
    plt.show()
    plt.close()

In [13]:
def get_real_samples(X_N, y_N, X_S, y_S, X_V, y_V):
    
    # choose random instances
    i_N = randint(0, y_N.shape[0], 1)
    i_S = randint(0, y_S.shape[0], 1)
    i_V = randint(0, y_V.shape[0], 1)
    
    # select ECG and labels
    X = np.vstack((X_N[i_N], X_S[i_S], X_V[i_V]))
    return X

def save_new_plot(X, n, name):
    plt.figure(figsize=(15,5))
    for i in range(n-1):
        plt.subplot(n+1, n-1, 1 + i)
        plt.axis('off')
        plt.plot(X[i, :, 0])
    for i in range(3, (n*n)-n):
        # define subplot
        plt.subplot(n+1, n-1, 1 + i)
        # turn off axis
        plt.axis('off')
        # plot raw pixel data
        plt.plot(X[i, :, 0])
    plt.savefig(name, dpi=50)
    plt.close()

In [14]:
# size of the latent space
latent_dim = 99
# size of the data
data = 186
# classes
classes = 3

n_epochs=50

# multiples of three (three classes) (less thyan 24000)
n_batch=300

# Loss Values
D_L_1 = np.infty
D_L_2 = np.infty
G_L = np.infty

# samples to watch
n=3

# create the discriminator
d_model = discriminator(data_dim=data, input_classes=classes)
# create the generator
g_model = generator(noise_dim=latent_dim, input_classes=classes, out_dim=data)
# create the gan
gan_model = create_gan(d_model, g_model)

# load image data
X_N, y_N, X_S, y_S, X_V, y_V = load_real_samples()
# # train model
# train(g_model, d_model, gan_model, dataset, latent_dim)

bat_per_epo = int(y_S.shape[0] / n_batch)
half_batch = int(n_batch / 2)
flag=0

# losses = []
plt.ioff()

# filename = 'Concatenate_Sigmoid_OHE_LI'
# filename = 'Concatenate_TANH_OHE_LI'
filename = 'Multiply_Sigmoid_OHE_LI'
# filename = 'Multiply_Tanh_OHE_LI'

if not os.path.isdir(filename):
    os.mkdir(filename)

f = open(filename + '_Loss.csv', 'w')
f.write('d_loss1, d_loss2, g_loss \n')
f.close()

# manually enumerate epochs
for i in range(n_epochs):
    # enumerate batches over the training set
    for j in range(bat_per_epo):
        # get randomly selected 'real' samples
        [X_real, labels_real], y_real = generate_real_samples(X_N, y_N, X_S, y_S, X_V, y_V, half_batch)
        # print (X_real.shape, labels_real.shape, y_real.shape)
        # update discriminator model weights
        d_loss1, _, _, _, _ = d_model.train_on_batch([X_real, labels_real], [y_real, labels_real])
        # generate 'fake' examples
        [X_fake, labels], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
        # print (X_fake.shape, labels.shape, y_fake.shape)
        # update discriminator model weights
        d_loss2, _, _, _, _ = d_model.train_on_batch([X_fake, labels], [y_fake, labels])
        # prepare points in latent space as input for the generator
        [z_input, labels_input] = generate_latent_points(latent_dim, n_batch)
        # print (z_input.shape, labels_input.shape)
        # create inverted labels for the fake samples
        y_gan = np.random.uniform(0.7, 1, n_batch)
        y_gan = y_gan.reshape(y_gan.shape[0], 1)
        # y_gan = ones((n_batch, 1))
        # update the generator via the discriminator's error
        g_loss, _, _ = gan_model.train_on_batch([z_input, labels_input], [y_gan, labels_input])
        
        if (d_loss1 < D_L_1 and d_loss2 < D_L_2) or g_loss < G_L:
            D_L_1 = d_loss1
            D_L_2 = d_loss2
            G_L = g_loss
            g_model.save(filename + '_cgan_generator.h5')
        
#         losses.append((d_loss1, d_loss2, g_loss))
        f = open(filename + '_Loss.csv', 'a')
        f.write(str(d_loss1)+','+str(d_loss2)+','+str(g_loss)+'\n')
        f.close()
        
        # summarize loss on this batch
        if (j+1)%2 == 0:
            print('>%d, %d/%d, d1=%.5f, d2=%.5f g=%.5f' %(i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
        if (j+1)%5 == 0:
            name = filename + '/'+str(i*1000 + j)+'.jpg'
            # generate images
            latent_points, labels = generate_latent_points(latent_dim, n*n)
            # specify labels
            labels = asarray([x for _ in range(n) for x in range(n)])
            # generate images
            X  = g_model.predict([latent_points, keras.utils.to_categorical(labels)])
            X_R = get_real_samples(X_N, y_N, X_S, y_S, X_V, y_V)
            X = np.vstack((X_R, X))
            save_new_plot(X, n+1, name)

>1, 2/80, d1=0.83794, d2=0.76291 g=1.77240
>1, 4/80, d1=0.71520, d2=0.59706 g=1.76489
>1, 6/80, d1=0.60555, d2=0.52948 g=1.75849
>1, 8/80, d1=0.53440, d2=0.47093 g=1.75217
>1, 10/80, d1=0.49901, d2=0.45647 g=1.74735
>1, 12/80, d1=0.49119, d2=0.46097 g=1.74123
>1, 14/80, d1=0.47921, d2=0.42197 g=1.73408
>1, 16/80, d1=0.45931, d2=0.43574 g=1.72594
>1, 18/80, d1=0.46253, d2=0.43019 g=1.72008
>1, 20/80, d1=0.43754, d2=0.42741 g=1.71477
>1, 22/80, d1=0.45891, d2=0.44295 g=1.71439
>1, 24/80, d1=0.44486, d2=0.42609 g=1.70394
>1, 26/80, d1=0.44106, d2=0.44148 g=1.69996
>1, 28/80, d1=0.42055, d2=0.44632 g=1.68515
>1, 30/80, d1=0.44115, d2=0.44213 g=1.67866
>1, 32/80, d1=0.43636, d2=0.43949 g=1.67411
>1, 34/80, d1=0.43000, d2=0.42080 g=1.66772
>1, 36/80, d1=0.41076, d2=0.43030 g=1.65988
>1, 38/80, d1=0.42484, d2=0.43060 g=1.65011
>1, 40/80, d1=0.44188, d2=0.40978 g=1.64733
>1, 42/80, d1=0.42363, d2=0.42845 g=1.63494
>1, 44/80, d1=0.42265, d2=0.46811 g=1.62769
>1, 46/80, d1=0.46429, d2=0.41333 g=

>5, 56/80, d1=0.45665, d2=0.43460 g=0.42316
>5, 58/80, d1=0.44164, d2=0.41825 g=0.43930
>5, 60/80, d1=0.42386, d2=0.44614 g=0.41829
>5, 62/80, d1=0.42195, d2=0.43751 g=0.42835
>5, 64/80, d1=0.42277, d2=0.42316 g=0.43747
>5, 66/80, d1=0.41783, d2=0.43179 g=0.43924
>5, 68/80, d1=0.41251, d2=0.42394 g=0.42915
>5, 70/80, d1=0.40546, d2=0.41669 g=0.43988
>5, 72/80, d1=0.43929, d2=0.43046 g=0.44244
>5, 74/80, d1=0.44369, d2=0.43849 g=0.42674
>5, 76/80, d1=0.45117, d2=0.39811 g=0.43403
>5, 78/80, d1=0.41261, d2=0.43253 g=0.43698
>5, 80/80, d1=0.43782, d2=0.45029 g=0.45972
>6, 2/80, d1=0.42872, d2=0.43426 g=0.44113
>6, 4/80, d1=0.40124, d2=0.43698 g=0.42996
>6, 6/80, d1=0.42447, d2=0.43789 g=0.44027
>6, 8/80, d1=0.43065, d2=0.41255 g=0.43934
>6, 10/80, d1=0.42085, d2=0.45090 g=0.41875
>6, 12/80, d1=0.42256, d2=0.41804 g=0.44493
>6, 14/80, d1=0.42944, d2=0.41353 g=0.43598
>6, 16/80, d1=0.42092, d2=0.43668 g=0.42305
>6, 18/80, d1=0.44637, d2=0.41358 g=0.44283
>6, 20/80, d1=0.43911, d2=0.43813 g=

>10, 30/80, d1=0.43628, d2=0.41652 g=0.47095
>10, 32/80, d1=0.43076, d2=0.43227 g=0.47179
>10, 34/80, d1=0.41889, d2=0.43861 g=0.49083
>10, 36/80, d1=0.42487, d2=0.43517 g=0.46252
>10, 38/80, d1=0.43562, d2=0.42677 g=0.47118
>10, 40/80, d1=0.43161, d2=0.43756 g=0.48648
>10, 42/80, d1=0.43168, d2=0.42801 g=0.46387
>10, 44/80, d1=0.41444, d2=0.42699 g=0.45029
>10, 46/80, d1=0.41870, d2=0.43895 g=0.48341
>10, 48/80, d1=0.44745, d2=0.43662 g=0.48359
>10, 50/80, d1=0.42924, d2=0.42565 g=0.50047
>10, 52/80, d1=0.43931, d2=0.41021 g=0.47452
>10, 54/80, d1=0.40290, d2=0.40689 g=0.48521
>10, 56/80, d1=0.42643, d2=0.42433 g=0.50171
>10, 58/80, d1=0.42103, d2=0.44936 g=0.51600
>10, 60/80, d1=0.44854, d2=0.40970 g=0.47478
>10, 62/80, d1=0.42144, d2=0.43277 g=0.50481
>10, 64/80, d1=0.44055, d2=0.42437 g=0.48430
>10, 66/80, d1=0.41048, d2=0.42311 g=0.46397
>10, 68/80, d1=0.40494, d2=0.43189 g=0.48205
>10, 70/80, d1=0.42926, d2=0.43232 g=0.45576
>10, 72/80, d1=0.41757, d2=0.42825 g=0.45442
>10, 74/80

>14, 76/80, d1=0.44817, d2=0.41386 g=0.50059
>14, 78/80, d1=0.41389, d2=0.42369 g=0.47150
>14, 80/80, d1=0.40185, d2=0.45257 g=0.48677
>15, 2/80, d1=0.41476, d2=0.44178 g=0.46679
>15, 4/80, d1=0.43985, d2=0.43045 g=0.47558
>15, 6/80, d1=0.43326, d2=0.41999 g=0.51898
>15, 8/80, d1=0.42319, d2=0.43109 g=0.49498
>15, 10/80, d1=0.43084, d2=0.44825 g=0.49536
>15, 12/80, d1=0.42538, d2=0.42086 g=0.50221
>15, 14/80, d1=0.43206, d2=0.43836 g=0.50374
>15, 16/80, d1=0.42280, d2=0.41983 g=0.47681
>15, 18/80, d1=0.41665, d2=0.41628 g=0.48290
>15, 20/80, d1=0.45618, d2=0.43329 g=0.49338
>15, 22/80, d1=0.42113, d2=0.44303 g=0.49684
>15, 24/80, d1=0.44357, d2=0.44071 g=0.46685
>15, 26/80, d1=0.43940, d2=0.43611 g=0.49687
>15, 28/80, d1=0.41899, d2=0.44719 g=0.47374
>15, 30/80, d1=0.41513, d2=0.40561 g=0.47571
>15, 32/80, d1=0.42319, d2=0.42251 g=0.50242
>15, 34/80, d1=0.43580, d2=0.41161 g=0.47908
>15, 36/80, d1=0.39198, d2=0.43901 g=0.47319
>15, 38/80, d1=0.42994, d2=0.42514 g=0.48833
>15, 40/80, d1

>19, 42/80, d1=0.41959, d2=0.41875 g=0.48155
>19, 44/80, d1=0.41774, d2=0.41603 g=0.49107
>19, 46/80, d1=0.46713, d2=0.42947 g=0.48043
>19, 48/80, d1=0.40729, d2=0.43050 g=0.46485
>19, 50/80, d1=0.43664, d2=0.41971 g=0.47298
>19, 52/80, d1=0.44056, d2=0.45460 g=0.46491
>19, 54/80, d1=0.41029, d2=0.41052 g=0.50052
>19, 56/80, d1=0.45986, d2=0.44171 g=0.48353
>19, 58/80, d1=0.41737, d2=0.43365 g=0.47603
>19, 60/80, d1=0.43708, d2=0.43343 g=0.47243
>19, 62/80, d1=0.41435, d2=0.45088 g=0.46303
>19, 64/80, d1=0.41561, d2=0.40485 g=0.46693
>19, 66/80, d1=0.44065, d2=0.41155 g=0.48029
>19, 68/80, d1=0.43086, d2=0.43428 g=0.47047
>19, 70/80, d1=0.43134, d2=0.43512 g=0.45626
>19, 72/80, d1=0.42022, d2=0.42983 g=0.46241
>19, 74/80, d1=0.41466, d2=0.44520 g=0.46931
>19, 76/80, d1=0.42791, d2=0.43129 g=0.45784
>19, 78/80, d1=0.40482, d2=0.44020 g=0.47270
>19, 80/80, d1=0.42900, d2=0.41606 g=0.46843
>20, 2/80, d1=0.42192, d2=0.43716 g=0.47438
>20, 4/80, d1=0.42257, d2=0.41761 g=0.51222
>20, 6/80, d

...

>28, 54/80, d1=0.42528, d2=0.43552 g=0.48414
>28, 56/80, d1=0.43320, d2=0.40808 g=0.49201
>28, 58/80, d1=0.42415, d2=0.44565 g=0.47842
>28, 60/80, d1=0.42719, d2=0.43598 g=0.47564
>28, 62/80, d1=0.42877, d2=0.44957 g=0.46790
>28, 64/80, d1=0.42899, d2=0.42145 g=0.47939
>28, 66/80, d1=0.39552, d2=0.44312 g=0.47285
>28, 68/80, d1=0.42881, d2=0.41361 g=0.47798
>28, 70/80, d1=0.44383, d2=0.43190 g=0.48858
>28, 72/80, d1=0.42472, d2=0.42221 g=0.46959
>28, 74/80, d1=0.44045, d2=0.40307 g=0.52119
>28, 76/80, d1=0.40521, d2=0.43116 g=0.48777
>28, 78/80, d1=0.44082, d2=0.45608 g=0.47017
>28, 80/80, d1=0.42545, d2=0.43723 g=0.47540
>29, 2/80, d1=0.41846, d2=0.44603 g=0.46525
>29, 4/80, d1=0.40504, d2=0.42201 g=0.47825
>29, 6/80, d1=0.41628, d2=0.41724 g=0.45487
>29, 8/80, d1=0.43467, d2=0.40458 g=0.48841
>29, 10/80, d1=0.42794, d2=0.41194 g=0.48408
>29, 12/80, d1=0.39642, d2=0.43364 g=0.47863
>29, 14/80, d1=0.43844, d2=0.42573 g=0.47616
>29, 16/80, d1=0.42081, d2=0.42349 g=0.47820
>29, 18/80, d1

>33, 20/80, d1=0.42807, d2=0.40258 g=0.47520
>33, 22/80, d1=0.43170, d2=0.42171 g=0.46155
>33, 24/80, d1=0.41541, d2=0.40925 g=0.45448
>33, 26/80, d1=0.45178, d2=0.41836 g=0.48805
>33, 28/80, d1=0.42283, d2=0.41732 g=0.47809
>33, 30/80, d1=0.43639, d2=0.42099 g=0.49746
>33, 32/80, d1=0.44622, d2=0.42659 g=0.48889
>33, 34/80, d1=0.42346, d2=0.39895 g=0.47493
>33, 36/80, d1=0.42532, d2=0.40949 g=0.47020
>33, 38/80, d1=0.42730, d2=0.39516 g=0.46629
>33, 40/80, d1=0.42833, d2=0.41888 g=0.47086
>33, 42/80, d1=0.43052, d2=0.42415 g=0.48429
>33, 44/80, d1=0.42493, d2=0.43969 g=0.48069
>33, 46/80, d1=0.42414, d2=0.43039 g=0.48110
>33, 48/80, d1=0.41601, d2=0.42621 g=0.49159
>33, 50/80, d1=0.41822, d2=0.40772 g=0.49778
>33, 52/80, d1=0.41978, d2=0.42731 g=0.50372
>33, 54/80, d1=0.43408, d2=0.41806 g=0.50184
>33, 56/80, d1=0.45209, d2=0.45151 g=0.47072
>33, 58/80, d1=0.42804, d2=0.43659 g=0.49000
>33, 60/80, d1=0.43352, d2=0.42429 g=0.50407
>33, 62/80, d1=0.40038, d2=0.42391 g=0.48728
>33, 64/80

>37, 66/80, d1=0.42768, d2=0.44195 g=0.47696
>37, 68/80, d1=0.44788, d2=0.43510 g=0.50172
>37, 70/80, d1=0.40684, d2=0.41608 g=0.49791
>37, 72/80, d1=0.44717, d2=0.41454 g=0.49960
>37, 74/80, d1=0.41954, d2=0.41115 g=0.49180
>37, 76/80, d1=0.42084, d2=0.41541 g=0.48638
>37, 78/80, d1=0.42495, d2=0.43911 g=0.50503
>37, 80/80, d1=0.41862, d2=0.40016 g=0.50283
>38, 2/80, d1=0.41680, d2=0.41677 g=0.48535
>38, 4/80, d1=0.42771, d2=0.44418 g=0.47833
>38, 6/80, d1=0.43458, d2=0.42847 g=0.48727
>38, 8/80, d1=0.42262, d2=0.41355 g=0.50283
>38, 10/80, d1=0.41461, d2=0.43607 g=0.47419
>38, 12/80, d1=0.41894, d2=0.43002 g=0.47794
>38, 14/80, d1=0.41204, d2=0.43241 g=0.48613
>38, 16/80, d1=0.42669, d2=0.41792 g=0.49367
>38, 18/80, d1=0.42318, d2=0.43060 g=0.48777
>38, 20/80, d1=0.43339, d2=0.43152 g=0.50172
>38, 22/80, d1=0.42105, d2=0.42732 g=0.51005
>38, 24/80, d1=0.45449, d2=0.43175 g=0.51301
>38, 26/80, d1=0.42945, d2=0.43126 g=0.50822
>38, 28/80, d1=0.42015, d2=0.41924 g=0.53240
>38, 30/80, d1

>42, 32/80, d1=0.41378, d2=0.41502 g=0.47259
>42, 34/80, d1=0.41254, d2=0.42311 g=0.48108
>42, 36/80, d1=0.42916, d2=0.41555 g=0.48053
>42, 38/80, d1=0.43421, d2=0.43979 g=0.48743
>42, 40/80, d1=0.41694, d2=0.44493 g=0.47359
>42, 42/80, d1=0.43362, d2=0.41960 g=0.49162
>42, 44/80, d1=0.40906, d2=0.42361 g=0.48334
>42, 46/80, d1=0.42878, d2=0.42093 g=0.49834
>42, 48/80, d1=0.41896, d2=0.44186 g=0.48552
>42, 50/80, d1=0.42270, d2=0.41890 g=0.49495
>42, 52/80, d1=0.42668, d2=0.43330 g=0.48818
>42, 54/80, d1=0.41858, d2=0.42042 g=0.49974
>42, 56/80, d1=0.42609, d2=0.44533 g=0.48022
>42, 58/80, d1=0.44056, d2=0.43305 g=0.51460
>42, 60/80, d1=0.42084, d2=0.44893 g=0.48136
>42, 62/80, d1=0.41449, d2=0.43770 g=0.47657
>42, 64/80, d1=0.42605, d2=0.41582 g=0.51774
>42, 66/80, d1=0.42846, d2=0.41650 g=0.50763
>42, 68/80, d1=0.41844, d2=0.42081 g=0.49499
>42, 70/80, d1=0.41065, d2=0.41470 g=0.49010
>42, 72/80, d1=0.41094, d2=0.42291 g=0.51333
>42, 74/80, d1=0.41990, d2=0.44412 g=0.52150
>42, 76/80

>46, 78/80, d1=0.41935, d2=0.42764 g=0.47660
>46, 80/80, d1=0.41631, d2=0.41861 g=0.48194
>47, 2/80, d1=0.44057, d2=0.42657 g=0.47228
>47, 4/80, d1=0.42601, d2=0.40776 g=0.49406
>47, 6/80, d1=0.43408, d2=0.41331 g=0.49821
>47, 8/80, d1=0.42184, d2=0.41669 g=0.51783
>47, 10/80, d1=0.42389, d2=0.44794 g=0.50684
>47, 12/80, d1=0.43289, d2=0.42226 g=0.50582
>47, 14/80, d1=0.40201, d2=0.42401 g=0.50942
>47, 16/80, d1=0.40738, d2=0.42389 g=0.51179
>47, 18/80, d1=0.43019, d2=0.42410 g=0.49476
>47, 20/80, d1=0.42526, d2=0.40544 g=0.50836
>47, 22/80, d1=0.39984, d2=0.44322 g=0.48056
>47, 24/80, d1=0.43228, d2=0.44583 g=0.47818
>47, 26/80, d1=0.41754, d2=0.43532 g=0.48858
>47, 28/80, d1=0.44023, d2=0.41527 g=0.51282
>47, 30/80, d1=0.42311, d2=0.43127 g=0.52059
>47, 32/80, d1=0.42103, d2=0.42073 g=0.50441
>47, 34/80, d1=0.42473, d2=0.42404 g=0.52061
>47, 36/80, d1=0.43981, d2=0.43288 g=0.51383
>47, 38/80, d1=0.40917, d2=0.44511 g=0.49353
>47, 40/80, d1=0.42586, d2=0.43276 g=0.50621
>47, 42/80, d1

In [15]:
# from evaluation_metrics import *
# metric_to_calculate = ['FID', 'MMD', 'DTW', 'ED', 'PC', 'KLD', 'RMSE', 'TWED']
# f = open(data_dir+'Stats.csv', 'a')
# f.write('Epoch, d_loss1, d_loss2, g_loss, ')
# for i in range(len(label_dict)):
#     for mtc in metric_to_calculate:
#         f.write(str(mtc)+str(i)+',')
# f.write('Time,\n')
# f.close()

# d_loss1 = float(d_loss1.history['loss'][0])
# d_loss2 = float(d_loss2.history['loss'][0])
# g_loss = float(g_loss.history['loss'][0])

# if i%checkpoint == 0:
        
#         f = open(data_dir+'Stats.csv', 'a')
#         f.write(str(i+1)+','+str(d_loss1)+','+str(d_loss2)+','+str(g_loss)+',')
#         save_model(model=G, data_dir=data_dir, type='G', epoch=i)
#         save_model(model=D, data_dir=data_dir, type='D', epoch=i)
#         save_model(model=GAN, data_dir=data_dir, type='GAN', epoch=i)
        
#         for k,metric in enumerate(label_dict.keys()):    
#             temp_x = test_data[200*(k):200*(k+1),:-1]
#             [z_input, labels_input] = generate_class_specific_latent_input(200, n_classes=n_classes, noise_dim=noise_dim, category=float(metric))
#             z_input = G.predict([z_input, labels_input], verbose=verbose)

#             for j in range(2):
#                 plt.plot(z_input[j])
#             plt.savefig(data_dir+str(i)+'_Label_'+str(metric)+'.png')
#             plt.close()
#             plt.clf()
            
#             results = evaluate(temp_x,z_input,metric_to_calculate)
#             for r in results:
#                 f.write(str(r)+',')
            
#         f.write(str(end-start)+'\n')
#         f.close()

In [16]:
# #     # save the generator model
#     g_model.save('cgan_generator.h5')

# callback = [EarlyStopping(monitor='val_AUC', mode='max', verbose=1, patience=Pat),
#          ModelCheckpoint(filepath=str(twelve_lead_model_filename)+'_check_model.h5', 
#                          monitor='val_AUC', verbose=1, save_best_only=True, mode='max'),
#          ReduceLROnPlateau(monitor='val_AUC', factor=0.5, patience=Pat//2, verbose=1, 
#                            mode='max', min_delta=0.0001, cooldown=0, min_lr=0)]

# # model = parallel_NN(WINDOW_SIZE,INPUT_FEAT,OUTPUT_CLASS):
# model = parallel_NN(Window, len(leads), snomed_classes.shape[0])

# model.compile(loss=tf.keras.losses.BinaryCrossentropy(), 
#               optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
#               metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy', dtype=None, threshold=0.5),
#                        tf.keras.metrics.Recall(name='Recall'),
#                        tf.keras.metrics.Precision(name='Precision'),
#                        tf.keras.metrics.AUC(num_thresholds=200,summation_method="interpolation",
#                                             name="AUC",dtype=None,curve="ROC",thresholds=None,
#                                             multi_label=True,label_weights=None)])
# history = model.fit(train_generator, steps_per_epoch=train_samples, epochs=EP, verbose=1,
#                 validation_data=val_generator, validation_steps=val_samples, callbacks=callback)


# history_name = output_directory + '/' + twelve_lead_filename
# print (twelve_lead_model_filename, history_name)

# save_model(twelve_lead_model_filename, model)
# write_history(history_name, history.history)

In [17]:
# def wasserstein_loss(self, y_true, y_pred):
#     return K.mean(y_true * y_pred)